<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# FFE Breakdown `GiRaFFEfood` Initial Data for `GiRaFFE`

## This module provides another initial data option for `GiRaFFE`, drawn from [this paper](https://arxiv.org/abs/1310.3274) .

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation). The initial data has validated against the original `GiRaFFE`, as documented [here](Tutorial-Start_to_Finish_UnitTest-GiRaFFEfood_NRPy.ipynb).

### NRPy+ Source Code for this module: [GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_FFE_Breakdown.py](../../edit/in_progress/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_1D_tests_FFE_breakdown.py)

## Introduction:

### FFE Breakdown:

This is a flat-spacetime test with initial data 
\begin{align}
A_x &= 0, \\ 
A_y &= \left \{ \begin{array}{lll} x-0.2 & \mbox{if} & x<0 \\
           				  -5.0x^2 + x -0.2 & \mbox{if} & 0 < x < 0.2 \\
                                        -x & \mbox{if} & x>0.2         
                                        \end{array} \right. \\ 
A_z &= y - A_y
\end{align}
which generates the magnetic field in the wave frame,
\begin{align}
\mathbf{B}(0,x) = \left \{ \begin{array}{lll} (1.0,1.0,1.0) & \mbox{if} & x<0 \\
				                       	  (1.0,z(x),z(x)) & \mbox{if} & 0 < x < 0.2 \\
                                          (1.0,-1.0,-1.0) & \mbox{if} & x>0.2         
                                          \end{array}, \right.
\end{align}
where $z(x) = -10.0x+1.0$.

The electric field is then given by
$$\mathbf{E}(0,x) = (0.0,0.5,-0.5).$$

See the [Tutorial-GiRaFFEfood_NRPy_Exact_Wald](Tutorial-GiRaFFEfood_NRPy.ipynb) tutorial notebook for more general detail on how this is used.


<a id='toc'></a>

# Table of Contents:
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters 
1. [Step 2](#vector_ak): Set the vector $A_k$
1. [Step 3](#vectors_for_velocity): Set the vectors $B^i$ and $E^i$ for the velocity
1. [Step 4](#vi): Calculate $v^i$
1. [Step 5](#code_validation): Code Validation against `GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_1D_tests` NRPy+ module
1. [Step 6](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Import core NRPy+ modules and set NRPy+ parameters \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Here, we will import the NRPy+ core modules and set the reference metric to Cartesian, set commonly used NRPy+ parameters, and set C parameters that will be set from outside the code eventually generated from these expressions. We will also set up a parameter to determine what initial data is set up, although it won't do much yet.

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 0.a: Import the NRPy+ core modules and set the reference metric to Cartesian
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support

import reference_metric as rfm   # NRPy+: Reference metric support
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

# Step 1a: Set commonly used parameters.
thismodule = "GiRaFFEfood_NRPy_1D"

##### <a id='vector_ak'></a>

# Step 2: Set the vector $A_k$ \[Back to [top](#toc)\]
$$\label{vector_ak}$$

The vector potential is given as
\begin{align}
A_x &= 0, \\ 
A_y &= \left \{ \begin{array}{lll} x-0.2 & \mbox{if} & x<0 \\
           				  -5.0x^2 + x -0.2 & \mbox{if} & 0 < x < 0.2 \\
                                        -x & \mbox{if} & x>0.2         
                                        \end{array} \right. \\ 
A_z &= y - A_y
\end{align}

However, to take full advantage of NRPy+'s automated function generation capabilities, we want to write this without the `if` statements, replacing them with calls to `fabs()`. To do so, we will use the NRPy+ module `Min_Max_and_Piecewise_Expressions`.

In [2]:
# We'll use reference_metric.py to define x and y
x = rfm.xx_to_Cart[0]
y = rfm.xx_to_Cart[1]

Now, we can define the vector potential. We will rewrite $A_y$ to make use of the functions provided by `Min_Max_and_Piecewise_Expressions`. As shown below, we make sure that at each boundary, each $\leq$ is paired with a $>$. (This choice is arbitrary, we could just as easily choose $<$ and $\geq$.) This does not change the data since the function is continuous. However, it is necessary for the functions in `Min_Max_and_Piecewise_Expressions` to output the correct results.

\begin{align}
A_x &= 0, \\ 
A_y &= \left \{ \begin{array}{lll} x-0.2 & \mbox{if} & x \leq 0 \\
           				  -5.0x^2 + x -0.2 & \mbox{if} & 0 < x \leq 0.2 \\
                                        -x & \mbox{if} & x>0.2         
                                        \end{array} \right. \\ 
A_z &= y - A_y
\end{align}


In [3]:
AD = ixp.zerorank1(DIM=3)

import Min_Max_and_Piecewise_Expressions as noif
boundL = sp.sympify(0)
boundR = sp.Rational(1,5)

Ayleft = x - sp.Rational(1,5)
Aycenter = -sp.sympify(5)*x**2 + x - sp.Rational(1,5)
Ayright = -x

AD[0] = sp.sympify(0)
AD[1] = noif.coord_leq_bound(x,boundL)*Ayleft\
       +noif.coord_greater_bound(x,boundL)*noif.coord_leq_bound(x,boundR)*Aycenter\
       +noif.coord_greater_bound(x,boundR)*Ayright
AD[2] = y - AD[1]

<a id='vectors_for_velocity'></a>

# Step 3: Set the vectors $B^i$ and $E^i$ for the velocity \[Back to [top](#toc)\]
$$\label{vectors_for_velocity}$$

We will start by setting the auxiliary function $z(x) = -10.0x+1.0$.

In [4]:
zfunc = -sp.sympify(10)*x + sp.sympify(1) # Naming it like this to avoid any possible confusion with the coordinate.

Now, we will set the magnetic field
\begin{align}
\mathbf{B}(0,x) = \left \{ \begin{array}{lll} (1.0,1.0,1.0) & \mbox{if} & x<0 \\
				                       	  (1.0,z(x),z(x)) & \mbox{if} & 0 < x < 0.2 \\
                                          (1.0,-1.0,-1.0) & \mbox{if} & x>0.2         
                                          \end{array}, \right.
\end{align}
where $z(x) = -10.0x+1.0$ has been set above.


In [5]:
BU = ixp.zerorank1(DIM=3)

BU[0] = sp.sympify(1)
BU[1] = BU[2] = noif.coord_leq_bound(x,boundL)                                   * sp.sympify(1)\
               +noif.coord_greater_bound(x,boundL)*noif.coord_leq_bound(x,boundR)* zfunc\
               +noif.coord_greater_bound(x,boundR)                               * -sp.sympify(1)

Our expression for the electric field is
$$\mathbf{E}(0,x) = (0.0,0.5,-0.5).$$

In [11]:
EU = ixp.zerorank1()
EU[0] = sp.sympify(0)
EU[1] = sp.Rational(1,2)
EU[2] = -sp.Rational(1,2)

-1/2


<a id='vi'></a>

# Step 4: Calculate $v^i$ \[Back to [top](#toc)\]
$$\label{vi}$$

Now, we calculate $$\mathbf{v} = \frac{\mathbf{E} \times \mathbf{B}}{B^2},$$ which is equivalent to $$v^i = [ijk] \frac{E^j B^k}{B^2},$$ where $[ijk]$ is the Levi-Civita symbol and $B^2 = \gamma_{ij} B^i B^j$ is a trivial dot product in flat space.


In [7]:
LeviCivitaSymbolDDD = ixp.LeviCivitaSymbol_dim3_rank3()

B2 = sp.sympify(0)
for i in range(3):
    # In flat spacetime, gamma_{ij} is just a Kronecker delta
    B2 += BU[i]**2 # This is trivial to extend to curved spacetime

ValenciavU = ixp.zerorank1()
for i in range(3):
    for j in range(3):
        for k in range(3):
            ValenciavU[i] += LeviCivitaSymbolDDD[i][j][k] * EU[j] * BU[k] / B2

<a id='code_validation'></a>

# Step 5: Code Validation against `GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_1D_tests` NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the `GiRaFFE` Aligned Rotator initial data equations  we intend to use between
1. this tutorial and 
2. the NRPy+ [`GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_1D_tests.py`](../edit/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_1D_tests.py) module.



In [8]:
import GiRaFFEfood_NRPy.GiRaFFEfood_NRPy_1D_tests_FFE_breakdown as gfho
gfho.GiRaFFEfood_NRPy_1D_tests_FFE_breakdown()

def consistency_check(quantity1,quantity2,string):
    if quantity1-quantity2==0:
        print(string+" is in agreement!")
    else:
        print(string+" does not agree!")
        sys.exit(1)

print("Consistency check between GiRaFFEfood_NRPy tutorial and NRPy+ module:")

for i in range(3):
    consistency_check(ValenciavU[i],gfho.ValenciavU[i],"ValenciavU"+str(i))
    consistency_check(AD[i],gfho.AD[i],"AD"+str(i))

Consistency check between GiRaFFEfood_NRPy tutorial and NRPy+ module:
ValenciavU0 is in agreement!
AD0 is in agreement!
ValenciavU1 is in agreement!
AD1 is in agreement!
ValenciavU2 is in agreement!
AD2 is in agreement!


<a id='latex_pdf_output'></a>

# Step 6: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFEfood_NRPy_1D_tests.pdf](Tutorial-GiRaFFEfood_NRPy_1D_tests.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [9]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFEfood_NRPy_1D_tests",location_of_template_file=os.path.join(".."))

pdflatex: security risk: running with elevated privileges
pdflatex: security risk: running with elevated privileges
pdflatex: security risk: running with elevated privileges
Created Tutorial-GiRaFFEfood_NRPy_1D_tests.tex, and compiled LaTeX file to
    PDF file Tutorial-GiRaFFEfood_NRPy_1D_tests.pdf
